<i><b>Public AI</b></i>
<br>

# 고차원에서의 합성곱 연산

### _Objective_
* 대부분의 합성곱 연산은 4차원의 데이터를 다룹니다. 이를 어떻게 연산하는 지를 배워봅니다. <br>
* 합성곱 연산의 연산량을 줄여주는 풀링 연산에 대해 배워봅니다.<br> 
  


In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Model

## 1. 3차원 데이터 : 컬러 이미지
---

이전까지는 2차원 데이터를 다루는 합성곱 연산을 살펴보았습니다.
하지만 대부분의 영상 데이터는 색상이라는 정보가 있어, 3차원 데이터로 구성됩니다.
그리고 연산을 진행할 때에는 대부분 배치 단위로, 복수 개의 데이터를 한 번에 처리하는 식으로 진행됩니다.

![Imgur](https://i.imgur.com/2oq27A0.png)

## 2. 하나의 필터로 3차원 데이터 합성곱 진행하기
---

채널 방향으로 입력 데이터와 필터의 합성곱 연산을 수행하고, 그 결과 데이터를 더해 하나의 출력값으로 표현합니다. 아래의 3차원 데이터를 통해, 합성곱 연산이 어떤 식으로 진행되는지 살펴보도록 하겠습니다.

### (1) 3차원 데이터 구성하기

아래와 같이 3차원으로 구성된 입력 행렬을 구성하였습니다.<br>

![Imgur](https://i.imgur.com/CaJF4dE.png)

In [2]:
in0 = np.array([
    [1,4,2,0],
    [2,3,1,0],
    [3,1,2,3],
    [4,3,2,1]
])
in1 = np.array([
    [1,7,2,1],
    [3,2,1,9],
    [0,1,5,1],
    [4,6,2,4]
])
in2 = np.array([
    [1,5,2,1],
    [3,2,1,9],
    [0,1,3,1],
    [4,6,2,7]
])
color_input = np.stack([in0, in1, in2],axis=-1)
print("입력값의 형태 (img Height, image Width, Image channel) :({},{},{})".format(*color_input.shape))

입력값의 형태 (img Height, image Width, Image channel) :(4,4,3)


### (2) Filter 구성하기

![Imgur](https://i.imgur.com/GDvXoqr.png)

여기서 중요한 것은 입력값의 채널 수(C)와 필터의 채널 수(C)가 동일해야 한다는 것입니다.<br>
입력값의 채널에 따라 채널 별 필터가 존재하고, 각 채널 별로 합성곱 연산을 진행한 후, 하나의 값으로 출력됩니다.<br>

In [3]:
f0 = np.array([
    [2,0,1],
    [0,1,2],
    [1,0,2]
])
f1 = np.array([
    [0,1,3],
    [2,1,3],
    [4,1,2]
])
f2 = np.array([
    [3,2,1],
    [2,2,3],
    [0,0,1]
])
filters = np.stack([f0,f1,f2],axis=-1)
print("Filter의 형태 (filter height, filter width, filter channel) : ({},{},{})".format(*filters.shape))

Filter의 형태 (filter height, filter width, filter channel) : (3,3,3)


### (3) 합성곱 연산 진행하기

padding은 valid, stride은 1로 두고 연산하면 아래와 같습니다.

![Imgur](https://i.imgur.com/tdy6Pcw.png)

In [4]:
# Loop 구문을 통해 수행
outputs = #

outputs

(0,0) 번째 값 : 82
(0,1) 번째 값 : 118
(1,0) 번째 값 : 92
(1,1) 번째 값 : 126


array([[ 82., 118.],
       [ 92., 126.]])

### (4) Keras 로 합성곱 연산 진행하기 

In [5]:
inputs = Input(shape=(4,4,3))
conv_0 = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), 
                padding='valid', name='conv_0', use_bias=False)(inputs)
model = Model(inputs, conv_0)
res_filter = filters.reshape(3,3,3,1)
model.get_layer('conv_0').set_weights([res_filter])

model.predict(color_input.reshape(1,4,4,3))

array([[[[ 82.],
         [118.]],

        [[ 92.],
         [126.]]]], dtype=float32)

### (5) 블록으로 시각화하기

블록으로 시각화한다면, 아래와 같이 나타낼 수 있습니다.

![Imgur](https://i.imgur.com/uNUYhhx.png)

* 입력 특징맵의 형태 : $(h_{in},w_{in},c_{in})$
* 필터의 형태 : $(h_f,w_f,c_{in})$
* 출력 특징맵의 형태 : $(h_{out},w_{out})$

출력의 형태는 이전에 배운 것과 마찬가지로<br>
필터의 크기, Stride, padding으로 결정됩니다.

## 3. 복수개의 필터로 3차원 데이터 합성곱 진행하기

---

하나의 Filter은 하나의 특징을 추출할 수 있습니다. 영상을 파악하기 위해서는 복수개의 특징을 함께 추출해, 판단해야 합니다. 복수개의 필터는 아래와 같이 진행됩니다.

### (1) Filter 구성하기

하나의 필터는 $(h_f,w_f,c_{in})$의 형태로 구성됩니다. 이러한 필터들을 복수개로 구성해 쌓으면 아래와 같습니다.

In [6]:
filter_1 = np.array([
    [[2,0,1],[0,1,2],[1,0,2]],
    [[0,1,3],[2,1,3],[4,1,2]],
    [[3,2,1],[2,2,3],[0,0,1]]])
filter_2 = np.array([
    [[4,0,1],[0,0,4],[0,3,2]],
    [[6,1,2],[3,5,1],[2,3,2]],
    [[1,4,1],[1,3,1],[2,1,0]]])

filters = np.stack([filter_1,filter_2],axis=0)
print("Filter의 형태 (N, H, W, C) : ({},{},{},{})".format(*filters.shape))

Filter의 형태 (N, H, W, C) : (2,3,3,3)


In [7]:
# Loop 구문을 통해 수행
outputs = #

outputs

array([[[ 78., 101.],
        [ 92., 145.]],

       [[103., 133.],
        [114., 166.]]])

### (2) Keras 로 합성곱 연산 진행하기 

![Imgur](https://i.imgur.com/Q1wXw64.png)

In [8]:
inputs = Input(shape=(4,4,3))
conv_1 = Conv2D(filters=2, kernel_size=(3,3), strides=(1,1), 
                padding='valid', name='conv_1', use_bias=False)(inputs)

model = Model(inputs, conv_1)

In [9]:
res_filter = filters.reshape(3,3,3,2)

model.get_layer('conv_1').set_weights([res_filter])

res_input = color_input.reshape(1,4,4,3)
model.predict(res_input)

array([[[[ 99.,  86.],
         [155.,  91.]],

        [[164.,  87.],
         [156., 141.]]]], dtype=float32)

### (3) 블록으로 시각화하기

블록으로 시각화한다면, 아래와 같이 나타낼 수 있습니다.


![Imgur](https://i.imgur.com/ONua9B4.png)




* 입력 특징맵의 형태 : $(h_{in},w_{in},c_{in})$
* 필터의 형태 : $(n_f,h_f,w_f,c_{in})$
* 출력 특징맵의 형태 : $(h_{out},w_{out},n_f)$

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---